Import libraries and instanciate camera

In [2]:
import cv2
import numpy as np
import serial
import time
import imutils

In [8]:
# Calibration parameters - coordinates in HSV
# color system used to extract ball from the background.
# Provided file range.py should be used to determine these
lowBoundHSV = (0, 0, 255)
highBoundHSV = (255, 255, 255)

# Pixel rows in between which frame is cropped.
# Ball is detected within that cropped picture,
# which is later on stitched to entire image for displaying
lowFrameCutoff = 185
highFrameCutoff = 295

# Serial module for UART communication with MCU
ser = serial.Serial('COM5', 19200, timeout=1)

def sendValue(serial, val):
    """
    Function which sends passed value val over serial
    to MCU. Value is formatted in such a way that three
    digit val is sent directly, while two and one digit
    vals are prepended with 'A' and 'AA', because zeroes
    are interpreted as end of string.
    
    Arguments
    ---------
    serial - serial.Serial instance for UART communication
    val - value to be transmitted ([0, 999] range)
    
    Returns
    -------
    None
    """
    # Convert val to string and append with 'A's if needed
    # Also, terminate string with '\r\n'
    s = str(val)
    if len(s) == 1:
        s = 'AA' + s
    elif len(s) == 2:
        s = 'A' + s
    s += '\r\n'
    
    # Send over UART to MCU
    ser.write(s.encode('utf-8'))
    
    
def calibrate():
    """
    Aux function used to set the camera properly so it only
    sees the rail, in order to minimize distractions from 
    reflections around it
    
    Arguments
    ---------
    None
    
    Returns
    -------
    None
    """
    try:
        # Camera object
        cap = cv2.VideoCapture(0)

        while True:
            # Grab a frame from camera
            ret, frame = cap.read()
            # If grabbing was unsuccessful, try again
            if not ret:
                print('Problem connecting to camera!')
                continue
                
            # Cut off part of the frame within defined bounds, in order
            # to extract just the ball and the rail
            cropped = frame[lowFrameCutoff:highFrameCutoff, :, :]
            # Display what camera sees
            cv2.imshow('Camera', cropped)

            # Wait for ENTER to exit
            if cv2.waitKey(1) == 13:
                break
        
        # Once program is done, release resources
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()
    
    
def run():
    """
    Grabs frame from camera and tries to detect circles
    in it. Once a circle has been detected, horizontal
    pixel value of its center is sent via UART to MCU.
    Middle point of the rail is considered to be aroud 320th
    pixel.
    Hint:
    ----------------> x
    |
    |
    |
    |
    |
    \/
    y
    
    Arguments
    ---------
    None
    
    Returns
    -------
    None
    """
    try:
        # Camera object
        cap = cv2.VideoCapture(0)
        
        while True:
            # Grab a frame from camera
            ret, frame = cap.read()
            # If grabbing was unsuccessful, try again
            if not ret:
                print('Problem connecting to camera!')
                continue

            # Cut off part of the frame within defined bounds, in order
            # to extract just the ball and the rail
            cropped = frame[lowFrameCutoff:highFrameCutoff, :, :]
            # Do some image processing to make detecting ball easier
            blurred = cv2.GaussianBlur(cropped, (11, 11), 0)
            hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv, lowBoundHSV, highBoundHSV)
            mask = cv2.erode(mask, None, iterations=2)
            mask = cv2.dilate(mask, None, iterations=2)
            
            # Find contours in the BW mask and initialize the current
            # (x, y) center of the ball
            cnts = imutils.grab_contours(cv2.findContours(mask, cv2.RETR_EXTERNAL,
                                                          cv2.CHAIN_APPROX_SIMPLE))
            
            # Only proceed if at least one contour was found
            if len(cnts) > 0:
                # Find the largest contour in the mask, then use
                # it to compute the minimum enclosing circle and
                # center
                c = max(cnts, key = cv2.contourArea)
                ((center_x, center_y), radius) = cv2.minEnclosingCircle(c)
                center_x = int(center_x)
                center_y = int(center_y)
                radius = int(radius)
                
                # Only proceed if the radius meets size requirements
                if radius > 30 and radius < 100:
                    # Draw the circle and center on the original frame
                    # hint: add the offset of lowFrameCutoff pixels to vertical axis
                    cv2.circle(frame, (center_x, center_y + lowFrameCutoff), radius, (0, 255, 0), 2)
                    cv2.circle(frame, (center_x, center_y + lowFrameCutoff), 5, (0, 0, 255), -1)
                    # Display how many pixels is ball off from the center
                    err = center_x - 322
                    cv2.putText(frame, "Error: " + str(err), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    
                    # Send position of the center over UART
                    sendValue(ser, center_x)
            else:
                print("Ball not detected!")
                sendValue(ser, 322)
            
            
            # Display detected circles
            cv2.imshow('Camera', frame)

            # Wait for ENTER to exit
            if cv2.waitKey(1) == 13:
                break
        
        # Once program is done, release resources
        cap.release()
        cv2.destroyAllWindows()
        ser.close()
        
    except Exception as e:
        print(e)
        cap.release()
        cv2.destroyAllWindows()
        ser.close()

In [9]:
run()

In [6]:
calibrate()